In [36]:
import numpy as np 
import pandas as pd

In [37]:
df = pd.read_csv("fake_or_real_news.csv",index_col=[0])

In [38]:
df.index.reindex(target=df.index,method='ReindexMethod')

(Index([ 8476, 10294,  3608, 10142,   875,  6903,  7341,    95,  4869,  2909,
        ...
         8411,  6143,  3262,  9337,  8737,  4490,  8062,  8622,  4021,  4330],
       dtype='int64', length=6335),
 None)

In [39]:
df.reset_index(inplace=True)

In [40]:
df.drop(columns=['index'],inplace=True)

In [41]:
lb = []
for i in df['label']:
    if i == 'FAKE':
        lb.append(0)
    else:
        lb.append(1)

In [42]:
lbl = np.array(lb)

In [43]:
df['label'] = lbl

In [44]:
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1


In [45]:
x = df.drop('label',axis=1)

In [46]:
y = df['label']

In [47]:
x.shape,y.shape

((6335, 2), (6335,))

In [48]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [49]:
voc_size = 5000

In [50]:
msg = x.copy()

In [51]:
import nltk
import re
from nltk.corpus import stopwords

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []

for i in range(0,len(msg)):
    review = re.sub('[^a-zA-Z]',' ',msg['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('English')]
    review = ' '.join(review)
    
    
    corpus.append(review)

In [54]:
len(corpus)

6335

In [55]:
one_hot_rep = [one_hot(words,voc_size) for words in corpus]
one_hot_rep

[[2965, 4897, 660],
 [4410, 3964, 259, 3258, 296, 3387, 4832, 3578, 123, 2698, 4047],
 [1890, 594, 725, 4948, 858],
 [1756, 635, 3589, 1613, 148, 3692, 459, 4924],
 [4885, 2952, 2422, 4374, 1213],
 [4074, 2210],
 [4888, 3846, 4410, 1031, 878, 3844],
 [3941, 2171, 3425],
 [822, 698, 123, 2235, 4721, 4977, 3816],
 [743, 4587, 3836, 2952, 1002, 746, 4876, 894, 269],
 [4016, 2235, 776, 380, 4077, 951, 4897, 2235, 2283],
 [3278, 123, 2479, 804, 4415, 1618, 1849, 1758, 3192, 2425],
 [4439, 149, 1770, 4169, 1115, 1817, 1140, 801, 4047],
 [382, 2952, 2221, 364, 1073],
 [123, 204, 3993, 2906],
 [1170, 2771, 2745],
 [1906, 653, 4450, 4897, 3678, 2069, 3000, 2285, 3459],
 [4897, 2235, 1609, 2398, 2952, 1382, 420, 583, 3496, 3757, 218, 293, 1140],
 [743, 658, 4450, 3723],
 [4895, 4040, 1139, 4268, 4354, 3872, 2952],
 [420, 703, 123, 4086, 3854, 1209],
 [4305, 4493, 4593, 3667, 2917, 2148, 387, 914, 4452],
 [4897, 2235, 3836, 459, 895, 4122, 56],
 [2952, 486, 4891, 362, 2986, 3516, 979, 4081],
 [20

# Embedding representation

In [56]:
sent_len = 10

embedded_docs = pad_sequences(one_hot_rep,padding='pre',maxlen=sent_len)

In [57]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0, 2965, 4897,  660],
      dtype=int32)

In [58]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features))

model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, sent_len))
print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 10, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [59]:
len(embedded_docs),len(y)

(6335, 6335)

In [60]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [62]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=42)


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5628 - loss: 0.6623 - val_accuracy: 0.7537 - val_loss: 0.5044
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8456 - loss: 0.3671 - val_accuracy: 0.7594 - val_loss: 0.4854
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8956 - loss: 0.2520 - val_accuracy: 0.7690 - val_loss: 0.5820
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9302 - loss: 0.1781 - val_accuracy: 0.7599 - val_loss: 0.6811
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9580 - loss: 0.1163 - val_accuracy: 0.7441 - val_loss: 0.7779
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9706 - loss: 0.0869 - val_accuracy: 0.7518 - val_loss: 0.9995
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9772 - loss: 0.0652 - val_accuracy: 0.7451 - val_loss: 1.2202
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9816 - loss: 0.0506 - val_accur

In [70]:
y_probs = model.predict(X_test)
y_pred = (y_probs > 0.5).astype(int)


66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [71]:
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [72]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [73]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7360114777618364
Confusion Matrix:
 [[831 240]
 [312 708]]
